In [1]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import pandas as pd
import cv2
import os

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from data_load import PhoneNumberDataset
from data_load import Normalize, Gray_scale_Image, ToTensor, Gray_scale_to_Binary, BoundingBoxes, Inversion

data_transform = [Gray_scale_Image(), Inversion(), Gray_scale_to_Binary(), BoundingBoxes(), ToTensor()]

assert(data_transform is not None)

In [2]:
net = torch.load("data\models\model_basic\modelWith2EpochOperaAppliedOnEachDigitBatchSize320.pth")
net.eval()

Net(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
  (maxPool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (mpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1600, out_features=80, bias=True)
  (fc1_drop): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=80, out_features=10, bias=True)
)

In [3]:
img = cv2.imread("data\Real_World_Data\i3.jpg")
img_copy = img.copy()
print(f"shape of original image: {img.shape}")
for j, fx in enumerate(data_transform):
    img_copy = fx(img_copy)

shape of original image: (256, 1150, 3)


In [4]:
def net_sample_output(img):
    images = img
    images = images.type(torch.FloatTensor)
    output_digits = net(images)
    output_digits = torch.argmax(output_digits, dim=1)
    output_digits = output_digits.view(output_digits.size()[0], -1)
    return output_digits

In [5]:
phone_number = torch.reshape(net_sample_output(img_copy), (1 ,10))
cv2.namedWindow("Real image", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Real image",335, 2786)
cv2.imshow("Real image", img)
print(phone_number.numpy()[0])
cv2.waitKey(0)
cv2.destroyAllWindows()

[1 7 3 4 5 6 7 8 9 0]


## Visualizing preprocessed image

In [6]:
for j,digit in enumerate(img_copy):
    cv2.namedWindow(f"digit{j}", cv2.WINDOW_NORMAL)
    cv2.resizeWindow(f"digit{j}",100, 100)
    cv2.imshow(f"digit{j}", digit.data.numpy())
cv2.waitKey(0)
cv2.destroyAllWindows()